## **Environment Setup**
 We'll begin by installing necessary libraries, including transformers, datasets, seqeval, and others needed for token classification and model evaluation.

In [ ]:
!pip install transformers datasets seqeval sentences
!pip install pyarrow==14.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement sentences (from versions: none)
ERROR: No matching distribution found for sentences
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pyarrow<16.2.0a0,>=16.1.0, but you have pyarrow 14.0.1 which is incompatible.


## **Import Libraries**
Next, we import key libraries required for token classification, dataset management, and training.

In [ ]:
!pip install datasets
# Import Libraries
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset

  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.1
    Uninstalling pyarrow-14.0.1:
      Successfully uninstalled pyarrow-14.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pyarrow<16.2.0a0,>=16.1.

## **Upload Dataset**
We upload the dataset in .conll format that has already been labeled with entities (Product, Location, Price).

In [ ]:
# Upload Dataset
from google.colab import files
uploaded = files.upload()

Saving combined_dataset.conll to combined_dataset.conll


## **Read Dataset and Preprocess**
The dataset in .conll format is read, and tokens are checked for any mismatches between the tokens and labels.

In [ ]:
# Function to read the .conll file and check for mismatches
def read_conll_format(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        sentence = []
        label = []
        for line in file:
            if line.strip() == "":
                if sentence:  # Only append non-empty sentences
                    sentences.append(sentence)
                    labels.append(label)
                    sentence = []
                    label = []
            else:
                token, tag = line.split()
                sentence.append(token)
                label.append(tag)
    return sentences, labels

# Read the file
file_path = 'combined_dataset.conll'
sentences, labels = read_conll_format(file_path)

# Check for mismatches between tokens and labels
mismatches = []
for i, (sentence, label) in enumerate(zip(sentences, labels)):
    if len(sentence) != len(label):
        mismatches.append((i, len(sentence), len(label)))

if mismatches:
    print("Mismatches found:", mismatches)
else:
    print("No mismatches found!")


No mismatches found!


## **Label Normalization**
In this step, we map all label variations (e.g., B-PRICE, I-PRICE) to a consistent naming convention to avoid errors during training.

In [ ]:
# Define a mapping for label standardization
label_mapping = {
    "O":"o",
    "B-LOC": "b-loc",
    "I-LOC": "i-loc",
    "B-PRODUCT": "b-product",
    "I-PRODUCT": "i-product",
    "B-PRICE": "b-price",
    "B-price": "b-price",
    "I-PRICE": "i-price",
}
combined_dataset = {"labels": labels}
# Normalize labels based on the mapping
combined_dataset["labels"] = [
    [label_mapping.get(label, label) for label in label_list]
    for label_list in combined_dataset["labels"]
]


## **Tokenizer Setup and Dataset Preparation**
We use the XLM-Roberta tokenizer to process the Amharic text. We also convert string labels to corresponding integer IDs, which the model can process.

In [ ]:
# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")


label_to_id = {
    "o": 0,
    "b-loc": 1,
    "i-loc": 2,
    "b-product": 3,
    "b-prod": 3, # Added missing label
    "i-product": 4,
    "b-price": 5,
    "i-price": 6,
    # Add any other labels you have in your dataset here
}

# Function to convert string labels to their corresponding integer IDs
def convert_labels_to_ids(examples):
    return {"labels": [[label_to_id[label] for label in label_sequence] for label_sequence in examples["labels"]]}

# Tokenize and align labels with conversion to integers
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to words
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id of None. We set the label to -100 so they are ignored
            if word_idx is None:
                label_ids.append(-100)
            # Only label the first token of a word
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # Labels for subwords should be ignored
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Create a dataset dictionary for easier processing
from datasets import Dataset
dataset_dict = {"tokens": sentences, "labels": labels}
dataset = Dataset.from_dict(dataset_dict)

# First, convert the string labels to their integer IDs
dataset = dataset.map(convert_labels_to_ids, batched=True)

# Now apply the tokenizer and alignment function
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/47878 [00:00<?, ? examples/s]

Map:   0%|          | 0/47878 [00:00<?, ? examples/s]

## **Split Dataset**

In [ ]:
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
validation_dataset = split_dataset['test']


## **Define Training Arguments**
We define the arguments for training the NER model, such as batch size, learning rate, and evaluation strategy.

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',  # Log directory
    logging_steps=10,
    save_strategy="epoch",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## **Define Compute Metrics**

In [ ]:
!pip install evaluate
!pip install seqeval

import evaluate
import numpy as np

# Load the seqeval metric for token classification tasks
metric = evaluate.load("seqeval")
label_list = ["o", "b-loc", "i-loc", "b-product", "i-product", "b-price", "i-price"]

# Define a compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [label_list[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)


  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5bbcde8c2026d003da8a05cfb31208f1bb6e167e4aba4fee5fb2552eefc0d57b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


## **Train the model**

In [ ]:
# Initialize model
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))

# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,,Loc,Price,Product,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.010400,0.019542,"{'precision': 0.8750815394651011, 'recall': 0.8843111404087014, 'f1': 0.879672131147541, 'number': 6068}","{'precision': 0.9749928140270192, 'recall': 0.9803468208092485, 'f1': 0.9776624873901137, 'number': 3460}","{'precision': 0.9565217391304348, 'recall': 0.9513513513513514, 'f1': 0.953929539295393, 'number': 3145}","{'precision': 0.8547687861271677, 'recall': 0.8666666666666667, 'f1': 0.8606766096762459, 'number': 2730}",0.910299,0.916445,0.913361,0.994969
2,0.033700,0.009282,"{'precision': 0.9443349753694581, 'recall': 0.9477587343441002, 'f1': 0.9460437571969075, 'number': 6068}","{'precision': 0.9909988385598142, 'recall': 0.9864161849710983, 'f1': 0.988702201622248, 'number': 3460}","{'precision': 0.9781299524564184, 'recall': 0.9812400635930048, 'f1': 0.9796825396825397, 'number': 3145}","{'precision': 0.9322897706589006, 'recall': 0.9380952380952381, 'f1': 0.9351834946138398, 'number': 2730}",0.959510,0.961566,0.960537,0.997475
3,0.000700,0.007956,"{'precision': 0.9622672598451145, 'recall': 0.962425840474621, 'f1': 0.962346543626926, 'number': 6068}","{'precision': 0.9947643979057592, 'recall': 0.9884393063583815, 'f1': 0.9915917657291968, 'number': 3460}","{'precision': 0.979746835443038, 'recall': 0.9844197138314785, 'f1': 0.9820777160983346, 'number': 3145}","{'precision': 0.9636163175303197, 'recall': 0.9604395604395605, 'f1': 0.9620253164556962, 'number': 2730}",0.973356,0.972408,0.972882,0.998184


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-product seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-price seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-price seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-loc seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeli

TrainOutput(global_step=14364, training_loss=0.022145927235595577, metrics={'train_runtime': 5652.1102, 'train_samples_per_second': 20.33, 'train_steps_per_second': 2.541, 'total_flos': 1.0826212019003304e+16, 'train_loss': 0.022145927235595577, 'epoch': 3.0})

## **Model Evaluation**
The model is evaluated after training. Metrics such as F1-score, precision, and recall will be computed.

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-product seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: b-price seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-price seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: i-loc seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeli

{'eval_loss': 0.007955918088555336, 'eval__': {'precision': 0.9622672598451145, 'recall': 0.962425840474621, 'f1': 0.962346543626926, 'number': 6068}, 'eval_loc': {'precision': 0.9947643979057592, 'recall': 0.9884393063583815, 'f1': 0.9915917657291968, 'number': 3460}, 'eval_price': {'precision': 0.979746835443038, 'recall': 0.9844197138314785, 'f1': 0.9820777160983346, 'number': 3145}, 'eval_product': {'precision': 0.9636163175303197, 'recall': 0.9604395604395605, 'f1': 0.9620253164556962, 'number': 2730}, 'eval_overall_precision': 0.9733558617104237, 'eval_overall_recall': 0.9724079724728949, 'eval_overall_f1': 0.9728816862070085, 'eval_overall_accuracy': 0.9981839807021333, 'eval_runtime': 120.6055, 'eval_samples_per_second': 79.399, 'eval_steps_per_second': 9.925, 'epoch': 3.0}


## **Save the Fine-tuned Model**
Finally, we save the fine-tuned model and tokenizer for future use.

In [ ]:
# Save the model
trainer.save_model("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/sentencepiece.bpe.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')